In [1]:
#import dependencies
import numpy as np
import pandas as pd
import os
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
#from imblearn.over_sampling import SMOTE
#!pip install fancyimpute
#from fancyimpute import IterativeImputer

from sklearn.mixture import GaussianMixture
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',None)
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
# from sklearn.inspection import permutation_importance

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

import tensorflow as tf

In [2]:
# load dataset
weatherdata = pd.read_csv('/kaggle/input/formulaaihackathon2022/weather.csv')

# EDA and Preprocessing

In [3]:
#copy of wetherdata
df = weatherdata.copy()

#drop columns(either based on variance or outliers)
irrelavant_features = ['M_PACKET_FORMAT', 'M_GAME_MAJOR_VERSION', 'M_GAME_MINOR_VERSION', 'M_PACKET_VERSION', 
                       'M_PACKET_ID', 'M_FRAME_IDENTIFIER', 'M_SECONDARY_PLAYER_CAR_INDEX', 'M_BRAKING_ASSIST', 'M_SESSION_LINK_IDENTIFIER', 
                       'M_PIT_RELEASE_ASSIST', 'M_ZONE_START', 'M_ZONE_FLAG', 'M_GAME_PAUSED', 'M_SLI_PRO_NATIVE_SUPPORT', 
                       'M_SAFETY_CAR_STATUS', 'M_ERSASSIST','M_FORMULA', 'M_PIT_ASSIST', 'M_GEARBOX_ASSIST', 'M_SPECTATOR_CAR_INDEX', 'M_DYNAMIC_RACING_LINE_TYPE', 
                       'M_  PIT_SPEED_LIMIT', 'M_NETWORK_GAME', 'M_STEERING_ASSIST', 'M_IS_SPECTATING', 'M_DYNAMIC_RACING_LINE', 
                       'M_DRSASSIST', 'M_NUM_MARSHAL_ZONES','GAMEHOST','Unnamed: 58']

df.drop(columns=irrelavant_features,inplace=True)

#split the feats into discrete and continous
cont_feats = [feat for feat in df.columns if df[feat].nunique()>50]
cat_feats = [feat for feat in df.columns if feat not in cont_feats]

In [13]:
def missing_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Zero_Values', 1 : 'Missing_Values', 2 : 'Missing_Values%'})
        mz_table['Sum_Zero_Plus_Missing_Values'] = mz_table['Zero_Values'] + mz_table['Missing_Values']
        mz_table['%_Sum_Zero_Plus_Missing_Values'] = 100 * mz_table['Sum_Zero_Plus_Missing_Values'] / len(df)
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        'Missing_Values%', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
#         mz_table.to_excel('D:/sampledata/missing_and_zero_values.xlsx', freeze_panes=(1,0), index = False)
        return mz_table

missing_values_df = missing_values_table(df)
missing_values_df

Since we have 27.3% of missing values in 8 Features, we shall use Multiple Imputations using Chained Equations - MICE to impute the values

In [ ]:
# # df1 = df.copy()
# # corr = df.corr()
# # plt.figure(figsize=(15,15))
# # sns.heatmap(corr,annot=True)

# f, ax = plt.subplots(3, 3, figsize = (40 , 20))
# axx = ax.flatten()

# for index, col in enumerate(missing_values_df.index.values):
#     sns.distplot(df[col],ax=axx[index])

In [4]:
#fill null values with ffill or agregated mean by date and time
df = df.fillna(method = 'ffill')

In [5]:
df['TIMESTAMP'] = df['TIMESTAMP'].apply(lambda x:datetime.fromtimestamp(x))
df['date'] =  df['TIMESTAMP'].dt.date
df['hour'] =  df['TIMESTAMP'].dt.hour
df['min'] =  df['TIMESTAMP'].dt.minute
df['sec'] =  df['TIMESTAMP'].dt.second

#sort values by date
df.sort_values(by=['TIMESTAMP'],inplace=True)

In [6]:
#copy of df 
dfcopy = df.copy()

#aggregate df by  M_SESSION_UID - M_SESSION_TIME - M_TIME_OFFSET
df_agg = df.groupby(['M_SESSION_UID','M_SESSION_TIME','M_TIME_OFFSET']).mean().reset_index()

In [7]:
df_agg

In [ ]:
# df_agg['M_WEATHER'][value_countsalue_countsEATHER']==0.5] = np.nan
# df_agg = df_agg.fillna(method = 'ffill')

# df[df['M_WEATHER_FORECAST_SAMPLES_M_SESSION_TYPE'].isnull()].tail(50)
# df[df['M_WEATHER_FORECAST_SAMPLES_M_SESSION_TYPE'].notnull()].tail(50)

## Random Forest

In [11]:
target = 'M_WEATHER'
# df_agg = df_agg.drop(columns=['date'])

X = df_agg.drop(columns=target)
y = df_agg[target]
X_train, X_test, y_train, y_test = train_test_split(X, y,shuffle=False,random_state=42)
#random forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

In [14]:
#predict for train and validation data
pred_train = rf.predict(X_train)
pred_test = rf.predict(X_test)

In [28]:
def metrics_print(actual,predicted, data_set):
    print(f'{data_set}')
    print('ACCURACY',accuracy_score(actual,predicted))
    print('RECALL', recall_score(actual,predicted,average='weighted'))
    print('PRECISION',precision_score(actual,predicted,average='weighted'))
    print('F1-SCORE',f1_score(actual,predicted,average='weighted'))

In [32]:
metrics_print(y_train,pred_train,'train')
metrics_print(y_test,pred_test,'test')

In [ ]:
# for feat in df_agg.columns:
#     print(feat,df_agg[feat].nunique())

In [35]:
discrete_feats = [col for col in df_agg.columns if df_agg[col].nunique() <25]
cont_feats = [col for col in df_agg.columns if col not in discrete_feats]

In [34]:
#discrete features
f, ax = plt.subplots(4, 4, figsize = (15 , 15))
axx = ax.flatten()

for index, col in enumerate(discrete_feats):
    df_agg[ col].value_counts().plot.pie(autopct = '%1.1f%%', ax = axx[index])

In [36]:
#bivariate analysis of categorical features with target
f, ax = plt.subplots(6, 3, figsize = (20 , 15))
axx = ax.flatten()

for index, col in enumerate(discrete_feats):
    sns.countplot(data = df_agg, x = col, hue = target, ax = axx[index])

In [37]:
#bivariate analysis of categorical features with target
f, ax = plt.subplots(5, 3, figsize = (20,20))
axx = ax.flatten()
for index, col in enumerate(cont_feats):
#     plt.figure(figsize=(15,8))
    sns.distplot(df_agg[col],ax=axx[index])
#     plt.show()

## Feature Engineering

In [8]:
#mean TRACK TEMP AND AIR TEMP per session id and hour
feats_gr  = ['M_SESSION_UID','hour']
feats_agg = ['M_TRACK_TEMPERATURE','M_AIR_TEMPERATURE']
dftemp = df_agg.groupby(feats_gr)[feats_agg].agg([np.mean])
cols = dftemp.columns
dftemp.columns = ['_SessionID_Hour_'.join(c) for c in cols]
dftemp.reset_index(inplace = True)
# join df with df_agg
df_agg = df_agg.merge(dftemp, on = feats_gr, how = 'left')

#drop features by variance
dropfeats = ['M_PIT_STOP_WINDOW_IDEAL_LAP','M_FORECAST_ACCURACY']

In [9]:
# session features 
sess_feats = ['M_SESSION_UID','M_SESSION_TYPE','M_SESSION_TIME','M_SESSION_TIME_LEFT','M_SESSION_DURATION']

### GMM

In [40]:
#get high correlated features for GMM model
highcorrelationfeats = ['M_AIR_TEMPERATURE', 'M_TRACK_TEMPERATURE_SessionID_Hour_mean','M_TRACK_TEMPERATURE']
tempdf = df_agg[highcorrelationfeats]

#Fit data using GMM
gmm = GaussianMixture(n_components=6, random_state = 1)
gmm.fit(tempdf)
tempdf['GMM_CLASS'] = gmm.predict(tempdf)


In [33]:
tempdf

In [43]:
tempdf['GMM_CLASS'].value_counts()

In [44]:
tempdf.groupby(['GMM_CLASS'])['M_TRACK_TEMPERATURE'].median().sort_values(ascending=False)

In [48]:
fig = plt.figure(figsize = (5,5))
sns.set(style = 'whitegrid')

sns.barplot(tempdf.GMM_CLASS, tempdf.M_TRACK_TEMPERATURE)

In [16]:
fig = plt.figure(figsize = (5,5))
sns.set(style = 'whitegrid')

sns.barplot(df_agg.M_WEATHER, df_agg.M_TRACK_TEMPERATURE)

In [67]:
 for feat in tempdf.columns:
     sns.displot(data=tempdf, x=feat, hue=df_agg['M_WEATHER'],
                 kind='kde',palette=sns.color_palette('bright')[:4],height=4,aspect=2.2)
     plt.show()

In [49]:
#map gmm class
gmmlabel = list(tempdf.groupby(['GMM_CLASS'])['M_TRACK_TEMPERATURE'].median().sort_values(ascending=False).index.values)
print('GMM_CLASS:', gmmlabel)


weatherlabel = [0,1,2,3,4,5]
print('M_WEATHER:',weatherlabel)

In [50]:
#map function to map GMM_CLASS to M_WEATHER
gmm_map = dict(zip(gmmlabel,weatherlabel))
print(gmm_map)

In [51]:
#map gmm class
df_agg['WEATHER_CLASS'] = tempdf['GMM_CLASS'].map(gmm_map)

In [62]:
sess_feats

In [71]:
#df_agg[df_agg['M_SESSION_UID']==df_agg['M_SESSION_UID'][0]]['M_SESSION_TIME_LEFT'].value_counts()

In [53]:
df_agg['WEATHER_CLASS'].value_counts()

In [ ]:
# for feat in tempdf.drop(columns='M_WEATHER').columns:
#     sns.displot(data=tempdf, x=feat, hue='M_WEATHER',
#                 kind='kde',palette=sns.color_palette('bright')[:4],height=4,aspect=2.2)
#     plt.show()

In [55]:
target = 'WEATHER_CLASS'
# df_agg = df_agg.drop(columns=['date'])

X = df_agg.drop(columns=['WEATHER_CLASS','M_WEATHER'])
y = df_agg[target]
X_train, X_test, y_train, y_test = train_test_split(X, y,shuffle=False,random_state=786)
#random forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=786)
rf.fit(X_train, y_train)

In [56]:
#predict for train and validation data
pred_train = rf.predict(X_train)
pred_test = rf.predict(X_test)

metrics_print(y_train,pred_train,'train')
metrics_print(y_test,pred_test,'test')

In [57]:
con_df = pd.DataFrame(list(zip(y_test.values,pred_test)),columns=['y_Actual','y_Predicted'])
confusion_matrix = pd.crosstab(con_df['y_Actual'], con_df['y_Predicted'], rownames=['Actual'], colnames=['Predicted'])
print (confusion_matrix)

In [58]:
con_df.value_counts()

## TabNet Classifier

In [59]:
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

!pip install pytorch-tabnet
from pytorch_tabnet.tab_model import TabNetClassifier

In [115]:
cols = df_agg.columns.to_list()
cols.remove('WEATHER_CLASS')
cols.remove('M_WEATHER')
cols

In [109]:
finaldf = df_agg.copy()
finaldf.drop(columns=['M_WEATHER'], inplace = True)

In [110]:
finaldf.columns

In [34]:
#X=df_agg.drop(columns=['WEATHER_CLASS'],inplace=False)
#Y=df_agg['WEATHER_CLASS']

In [116]:
from sklearn.model_selection import train_test_split

training_data, testing_data = train_test_split(finaldf, test_size=0.3, random_state=25, shuffle = False)

print(f"No. of training examples: {training_data.shape[0]}")
print(f"No. of testing examples: {testing_data.shape[0]}")

In [117]:
test = testing_data.copy()
test.drop(columns=['WEATHER_CLASS'], inplace = True)

In [99]:
training_data.shape

In [118]:
data = training_data.copy()

In [121]:
while True: pass

In [122]:
data_preds

In [ ]:
N_folds = 5
seeds = [455, 485, 659, 16, 5659]

data_preds = np.zeros((data.shape[0], 6))
test_preds = np.zeros((test.shape[0], 6))

feature_importance = pd.DataFrame()

for seed in seeds:
    print('---------------------------------------')
    print('SEED : ', seed)
    print('---------------------------------------')
    
    
    data['fold'] = -1
    skf = StratifiedKFold(n_splits=N_folds, shuffle=False, random_state=seed)
    for f, (_, idxs) in enumerate(skf.split(data, data['WEATHER_CLASS'])):
        data.loc[idxs, 'fold'] = f
        
    for F in range(N_folds):
        
        print('FOLD : ', F)
        
        train = data[data['fold'] != F].reset_index(drop=True)
        valid = data[data['fold'] == F].reset_index(drop=True)
        
        
        model = TabNetClassifier(verbose=0,seed=42)
        model.fit(X_train=train[cols].values, y_train=train['WEATHER_CLASS'].values,
                  eval_set=[(train[cols].values, train['WEATHER_CLASS'].values), (valid[cols].values, valid['WEATHER_CLASS'].values)],
                  eval_name=['train', 'valid'],
                 patience=5, max_epochs=100,
                 eval_metric=['accuracy'],
                 batch_size=1024, virtual_batch_size=256)
        
        
        preds = model.predict_proba(valid[cols].values)
        
        preds = model.predict_proba(test[cols].values)
        test_preds += preds

In [ ]:
# df_agg.groupby(['M_SESSION_UID'])['M_SESSION_TIME','M_SESSION_TIME_LEFT'].agg('min','max')

In [ ]:
# weatherdata.sort_values(by=['M_SESSION_UID','M_SESSION_TIME']).groupby(['M_SESSION_UID','M_SESSION_TIME']).mean().head()

In [ ]:
# df_agg.head(50)


In [ ]:
#df[df['M_SESSION_UID']==df_agg['M_SESSION_UID'][0]]

In [ ]:
#temp1 = weatherdata.sort_values(['M_SESSION_UID','M_SESSION_TIME'])

In [ ]:
#temp = weatherdata[(weatherdata['M_SESSION_UID']==weatherdata['M_SESSION_UID'][(weatherdata['M_WEATHER']!=0)][3089774]) ]

In [ ]:
#weatherdata['M_SESSION_UID'][(weatherdata['M_WEATHER']!=0)][3089774]

In [ ]:
# temp.sort_values(['M_SESSION_UID','M_SESSION_TIME']).head(50)

In [ ]:
#temp_agg = df.groupby(['M_SESSION_UID','M_SESSION_TIME','M_TIME_OFFSET','M_WEATHER']).mean().reset_index()

In [ ]:
#weatherdata['M_FORECAST_ACCURACY'].value_counts()

In [ ]:
#temp_agg['M_FORECAST_ACCURACY'].value_counts(normalize=True)

In [ ]:
#temp_agg[temp_agg['M_FORECAST_ACCURACY']==1]['M_WEATHER'].value_counts()

In [ ]:
#temp_agg['M_WEATHER'].value_counts()

In [ ]:
#temp_agg.head(50)

In [ ]:
#weatherdata.columns

In [ ]:
# weatherdata.groupby(['M_SESSION_UID','M_TIME_OFFSET']).count().head(50)

In [ ]:
# weatherdata.groupby(['M_SESSION_UID','TIMESTAMP']).median().reset_index()['M_TIME_OFFSET'].value_counts()

In [ ]:
#temp_agg.head(50)